In [88]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
from multiprocessing import Pool
from vtk.util import numpy_support
import pickle
import pandas as pd

In [85]:
def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()

def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    return x + dimx*(y+dimy*z)
    
def extract_and_dump_feature_vti(single_feature,data):
    
    dims = data.GetDimensions()
    gbounds = data.GetBounds()
    numPts = dims[0]*dims[1]*dims[2]
    
    feature_field = vtk.vtkImageData()
    feature_field.SetDimensions(dims)
    feature_field.SetExtent(data.GetExtent())
    feature_field.SetSpacing(data.GetSpacing())
    
    field = vtk.vtkDoubleArray()
    field.SetNumberOfTuples(numPts)
    field.SetName('feature_similarity')
    
    for i in range(numPts):
        field.SetTuple1(i,0.3)
   
    numFeaturePts = single_feature.GetNumberOfPoints()
    
    for i in range(numFeaturePts):
        pts = single_feature.GetPoint(i)
        xval = int(((pts[0] - gbounds[0])/(gbounds[1]-gbounds[0]))*dims[0])
        yval = int(((pts[1] - gbounds[2])/(gbounds[3]-gbounds[2]))*dims[1])
        zval = int(((pts[2] - gbounds[4])/(gbounds[5]-gbounds[4]))*dims[2])
        
        if xval > dims[0]-1:
            xval = dims[0]-1
        
        if yval > dims[1]-1:
            yval = dims[1]-1
            
        if zval > dims[2]-1:
            zval = dims[2]-1    
            
        index = compute_3d_to_1d_map(xval,yval,zval,dims[0],dims[1],dims[2])
        
        if index > dims[0]*dims[1]*dims[2] or index < 0:
            print (xval,yval,zval)
        
        val = data.GetPointData().GetArray('feature_similarity').GetTuple1(index)
        field.SetTuple1(index,val)
        
        if xval > dims[0] or yval >dims[1] or zval > dims[2]:
            print (xval,yval,zval)
            
    field.SetTuple1(0,0.0)       
    feature_field.GetPointData().AddArray(field)
        
    return feature_field
    

def segment_feature(fname,confidence_th,size_threshold, tstep, outpath):

    print ('processing time step: ' + str(tstep))
    
    idx=0
    cnt=0
        
    data = read_vti(fname)
    data.GetPointData().SetActiveScalars('feature_similarity')    
    gbounds = data.GetBounds()

    thresholding = vtk.vtkThreshold()
    thresholding.ThresholdByUpper( confidence_th )
    thresholding.SetInputData(data)
    seg = vtk.vtkConnectivityFilter()
    seg.SetInputConnection(thresholding.GetOutputPort())
    seg.SetExtractionModeToLargestRegion()
    seg.Update()

    segmentation = vtk.vtkConnectivityFilter()
    segmentation.SetInputConnection(thresholding.GetOutputPort())
    segmentation.SetExtractionModeToAllRegions()
    segmentation.ColorRegionsOn()
    segmentation.Update()

    ug = segmentation.GetOutput()
    num_segments = segmentation.GetNumberOfExtractedRegions()
    
    ## compute volumes of each bubble:
    bubble_volumes = np.zeros(num_segments)
    for i in range(ug.GetPointData().GetArray('RegionId').GetNumberOfTuples()):
        regionId = int(ug.GetPointData().GetArray('RegionId').GetTuple(i)[0])
        bubble_volumes[regionId] = bubble_volumes[regionId]+1     
    
    idx=0
    find_max_topmost_xvals=[]
    for i in range(num_segments):
        if  bubble_volumes[i] > size_threshold:
            thresholding2 = vtk.vtkThreshold()
            thresholding2.SetInputData(ug)
            thresholding2.ThresholdBetween(i,i)
            thresholding2.Update()
            single_feature = thresholding2.GetOutput()
            feature_pts = numpy_support.vtk_to_numpy(single_feature.GetPoints().GetData())
            max_x = np.max(feature_pts[:,0])
            find_max_topmost_xvals.append([max_x,i,idx])
            idx=idx+1
            
    ## identify the indices of features which are actually topmost and not desired bubbles
    find_max_topmost_xvals = np.asarray(find_max_topmost_xvals)
    top_indices=[]
    for i in range(len(find_max_topmost_xvals)):
        diff = np.abs(gbounds[1]-find_max_topmost_xvals[i,0])
        if diff < 0.00005:
            top_indices.append(int(find_max_topmost_xvals[i,1]))
    
    ## filter out the top most undesired features
    idx=0
    for i in range(num_segments):
        if  bubble_volumes[i] > size_threshold and i not in(top_indices):
            thresholding2 = vtk.vtkThreshold()
            thresholding2.SetInputData(ug)
            thresholding2.ThresholdBetween(i,i)
            thresholding2.Update()
            single_feature = thresholding2.GetOutput()
            idx=idx+1
            
            feature_field = extract_and_dump_feature_vti(single_feature,data)
            
            outfname = outpath + 'segmented_feature_' + str(i) + '_' + str(tstep) + '.vti'
            write_vti(outfname,feature_field)
            
        if  bubble_volumes[i] > size_threshold:
            cnt=cnt+1

In [83]:
## Parameters
init_time = 75
end_time = 408
input_path = '../out/mfix_local_grid/'
outpath = '../out/mfix_segmented_features/'
confidence_th = 0.9
size_threshold = 100

In [86]:
## Run for all timesteps
for ii in range(init_time,end_time):
    inpfname = input_path + 'slic_compare_' + str(ii) + '.vti'
    segment_feature(inpfname,confidence_th,size_threshold, ii, outpath)

processing time step: 75
processing time step: 76
processing time step: 77
processing time step: 78
processing time step: 79
processing time step: 80
processing time step: 81
processing time step: 82
processing time step: 83
processing time step: 84
processing time step: 85
processing time step: 86
processing time step: 87
processing time step: 88
processing time step: 89
processing time step: 90
processing time step: 91
processing time step: 92
processing time step: 93
processing time step: 94
processing time step: 95
processing time step: 96
processing time step: 97
processing time step: 98
processing time step: 99
processing time step: 100
processing time step: 101
processing time step: 102
processing time step: 103
processing time step: 104
processing time step: 105
processing time step: 106
processing time step: 107
processing time step: 108
processing time step: 109
processing time step: 110
processing time step: 111
processing time step: 112
processing time step: 113
processing 

processing time step: 392
processing time step: 393
processing time step: 394
processing time step: 395
processing time step: 396
processing time step: 397
processing time step: 398
processing time step: 399
processing time step: 400
processing time step: 401
processing time step: 402
processing time step: 403
processing time step: 404
processing time step: 405
processing time step: 406
processing time step: 407
